In [19]:
import torch
import pandas as pd
import numpy as np
import cv2
import os

In [9]:
# PC
data_dir = "G:/내 드라이브/data/FCComputerVision/DATASET/Segmentation/"
# colab
#data_dir = ""

In [7]:
def get_client_data(data_df, index):
    client_ids = np.unique(data_df.Id.values)
    client_id = client_ids[index]
    client_data = data_df[data_df.Id == client_id]
    image_files = list(client_data["ImageId"])
    mask_files = list(client_data["MaskId"])
    return client_id, image_files, mask_files

In [29]:
os.listdir(os.path.join(data_dir, 'train', 'images'))

['ID00255637202267923028520_29.jpg',
 'ID00275637202271440119890_92.jpg',
 'ID00381637202299644114027_221.jpg',
 'ID00111637202210956877205_35.jpg',
 'ID00123637202217151272140_71.jpg',
 'ID00186637202242472088675_291.jpg',
 'ID00068637202190879923934_22.jpg',
 'ID00015637202177877247924_119.jpg',
 'ID00322637202284842245491_61.jpg',
 'ID00015637202177877247924_12.jpg',
 'ID00192637202245493238298_79.jpg',
 'ID00170637202238079193844_1.jpg',
 'ID00102637202206574119190_224.jpg',
 'ID00219637202258203123958_316.jpg',
 'ID00381637202299644114027_383.jpg',
 'ID00032637202181710233084_24.jpg',
 'ID00167637202237397919352_100.jpg',
 'ID00073637202198167792918_69.jpg',
 'ID00068637202190879923934_81.jpg',
 'ID00076637202199015035026_5.jpg',
 'ID00075637202198610425520_92.jpg',
 'ID00129637202219868188000_26.jpg',
 'ID00035637202182204917484_20.jpg',
 'ID00009637202177434476278_386.jpg',
 'ID00111637202210956877205_256.jpg',
 'ID00075637202198610425520_47.jpg',
 'ID00290637202279304677843_209

In [20]:
class CT_dataset():
    def __init__(self, data_dir, phase, transformer=None):
        self.phase = phase
        self.images_dir = os.path.join(data_dir, phase, "images")
        self.masks_dir = os.path.join(data_dir, phase, "masks")
        self.image_files = [filename for filename in os.listdir(self.images_dir) if filename.endswith("jpg")]
        self.mask_files = [filename for filename in os.listdir(self.masks_dir) if filename.endswith("jpg")]
        assert len(self.image_files) == len(self.mask_files)
        
        self.transformer = transformer
    
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, index):
        image = cv2.imread(os.path.join(self.images_dir, self.image_files[index]))
        image = cv2.resize(image, dsize=(IMAGE_SIZE,IMAGE_SIZE), interpolation=cv2.INTER_LINEAR)
        mask = cv2.imread(os.path.join(self.masks_dir, self.mask_files[index]))
        mask = cv2.resize(mask, dsize=(IMAGE_SIZE,IMAGE_SIZE), interpolation=cv2.INTER_NEAREST)
        mask[mask < 240] = 0
        mask[mask >= 240] = 255
        mask = mask / 255
        
        mask_H, mask_W, mask_C = mask.shape
        background = np.ones(shape=(mask_H, mask_W))
        background[mask[...,0] != 0] = 0
        background[mask[...,1] != 0] = 0
        background[mask[...,2] != 0] = 0
        mask = np.concatenate([np.expand_dims(background, -1), mask], axis=-1)
        mask = np.argmax(mask, axis=-1, keepdims=False)
        
        if self.transformer:
            image = self.transformer(image)
        target = torch.from_numpy(mask).long()
        return image, target

In [1]:
from torchvision import transforms

def build_transformer():
    transformer = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize(mean=[485, 456, 406], std = [0.229, 224, 225])
    ])

C:\Users\PiusHwang\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
IMAGE_SIZE = 224

In [5]:
def collate_fn(batch):
    images = []
    targets = []
    for a, b in batch:
        images.append(a)
        targets.append(b)
    images = torch.stack(images, dim = 0)
    targets = torch.stack(targets, dim=0)
    return images, targets

In [31]:
print('data_dir: ', data_dir)
transformer = build_transformer()
dset = CT_dataset(data_dir=data_dir, phase= 'train', transformer=transformer)

data_dir:  G:/내 드라이브/data/FCComputerVision/DATASET/Segmentation/


In [17]:
image, target = dset[1]
print('image shape: {}'.format(image.shape))
print('target shape: {}'.format(target.shape))

error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4052: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [23]:
from torch.utils.data import DataLoader

In [24]:
dloader = DataLoader(dset, batch_size=4, shuffle=True, collate_fn=collate_fn)

In [ ]:
for index, batch in enumerate(dloader):
    images = batch[0]
    targets = batch[1]
    print('images shape: {}'.format(images.shape))
    print('targets shape: {}'.format(target.shape))

    if index == 0:
        break

In [35]:
def build_dataloader(data_dir, batch_size = 4):
    transformer = build_transformer()

    dataloaders = {}
    train_dataset = CT_dataset(data_dir=data_dir, phase='train', transformer=transformer)
    dataloaders['train'] = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

    val_dataset = CT_dataset(data_dir=data_dir, phase='val', transformer=transformer)
    dataloaders['val'] = DataLoader(val_dataset, batch_size=1, shuffle=False, collate_fn=collate_fn)
    return dataloaders

In [ ]:
print(data_dir)
dataloaders = build_dataloader(data_dir=data_dir, batch_size=4)

for phase in ['train', 'val']:
    for index, batch in enumerate(dataloaders[phase]):
        images = batch[0]
        targets = batch[1]
        print("image shape: {}".format(images.shape))
        print("targets shape: {}".format(targets.shape))

        if index == 0:
            break

# U-net Architecture 구현하기

![](https://miro.medium.com/max/700/1*f7YOaE4TWubwaFF7Z1fzNw.png)

* 512 x 512 image inputs
#### encoder
* feature map을 생성
* 이미지 크기를 줄이며 feature 차원을 늘림
* 이미지 크기를 점차 줄이며 resolution이 get lower
#### decoder
* encoder의 feature map 정보를 skip connection으로 받아 upsampling 하며 높은 정확도의 resolution의 sementic segmentation이 가능해짐
* up-conv convolution transpose: conv 연산한 결과 값 + conv fillter 값 으로 inverse matrix를 구현하는 것
* 즉 up-conv는 convolution operation을 하기 전에 이 결과 값을 토대로 conv operation의 입력 image를 보간하는 방법
* up-sampling 할 때 skip connection을 함: 

In [ ]:
import torch.nn as nn
def ConvLayer(in_channels, out_channels):
    layers = nn.Sequential(
        
    )